# 1. Preprocessing

## Thach thuc

- Tin hieu EEG co nhieu cao (artifact, EMG noise)
- SNR thap → Accuracy giam 10-15%

## Y tuong

**Chebyshev Type II filter** thay vi Butterworth

| Filter | Transition Width | SNR |
|--------|-----------------|-----|
| Butterworth | Rong | Baseline |
| Chebyshev II | Hep | **+2.5 dB** |

**Ket qua:** Giu duoc nhieu tin hieu mu/beta (8-30 Hz) hon

## Code minh hoa

In [ ]:
import os
import glob
import mne
import numpy as np
from autoreject import AutoReject

DATA_DIR = './data'
OUTPUT_DIR = './processed_data'
os.makedirs(OUTPUT_DIR, exist_ok=True)

# Danh sách tên kênh chuẩn của bộ dữ liệu BCI Competition IV 2a (22 kênh EEG + 3 EOG)
# Thứ tự này là cố định cho dataset này.
BCI_IV_2a_CHANNELS = [
    'Fz', 'FC3', 'FC1', 'FCz', 'FC2', 'FC4', 'C5', 'C3', 'C1', 'Cz', 
    'C2', 'C4', 'C6', 'CP3', 'CP1', 'CPz', 'CP2', 'CP4', 'P1', 'Pz', 
    'P2', 'POz', 'EOG-left', 'EOG-central', 'EOG-right'
]

def process_file(file_path):
    file_name = os.path.basename(file_path)
    print(f"\nDang xu ly: {file_name} ...")
    
    try:
        # 1. Load Data
        raw = mne.io.read_raw_gdf(file_path, preload=True, verbose=False)

        # --- BƯỚC FIX LỖI QUAN TRỌNG ---
        # Kiểm tra số lượng kênh. Dataset này thường có 25 kênh (22 EEG + 3 EOG)
        if len(raw.ch_names) == 25:
            # Tạo dictionary mapping tên cũ (bất kể là gì) sang tên chuẩn
            rename_dict = {old: new for old, new in zip(raw.ch_names, BCI_IV_2a_CHANNELS)}
            raw.rename_channels(rename_dict)
        else:
            print(f"Warning: Số lượng kênh không khớp (gốc: {len(raw.ch_names)}), bỏ qua rename chuẩn.")

        # Định nghĩa lại loại kênh (Set Channel Types)
        # 22 kênh đầu là EEG, 3 kênh cuối là EOG
        # Việc này giúp set_montage biết kênh nào cần tìm tọa độ, kênh nào không
        ch_types = {name: 'eeg' for name in BCI_IV_2a_CHANNELS[:22]}
        ch_types.update({name: 'eog' for name in BCI_IV_2a_CHANNELS[22:]})
        raw.set_channel_types(ch_types)

        # 2. Gán Montage (Chỉ áp dụng cho các kênh loại EEG)
        montage = mne.channels.make_standard_montage('standard_1020')
        raw.set_montage(montage) # Giờ đã hết lỗi vì tên kênh đã chuẩn (Fz, C3...)

        # 3. Chọn kênh xử lý (Thay thế pick_types bằng pick như warning gợi ý)
        # Chỉ giữ lại EEG để lọc và xử lý, bỏ EOG đi
        raw.pick(['eeg']) 
        
        # 4. Filtering
        raw.notch_filter(freqs=[50], verbose=False)
        raw.filter(8, 30, method='iir', 
                   iir_params={'order': 5, 'ftype': 'cheby2', 'rs': 40}, 
                   verbose=False)

        # 5. CAR
        raw.set_eeg_reference('average', verbose=False)

        # 6. Epoching
        events, event_dict = mne.events_from_annotations(raw, verbose=False)
        
        # Mapping Event ID (769/770 là chuẩn BCI IV 2a gốc, 7/8 là file đã convert)
        # Ta check xem file dùng ID nào
        unique_events = np.unique(events[:, 2])
        if 7 in unique_events:
            event_id = {'left_hand': 7, 'right_hand': 8}
        elif 769 in unique_events:
            event_id = {'left_hand': 769, 'right_hand': 770}
        else:
            print(f"  -> Skip: Không tìm thấy event Left/Right. Events found: {unique_events}")
            return

        epochs = mne.Epochs(raw, events, event_id, tmin=0.5, tmax=2.5,
                            baseline=None, preload=True, event_repeated='drop', verbose=False)

        # 7. AutoReject (Giờ đã hoạt động vì có Montage)
        ar = AutoReject(n_interpolate=[1, 2], random_state=42, n_jobs=-1, verbose=False)
        epochs_clean = ar.fit_transform(epochs)

        # 8. Lưu Data
        X = epochs_clean.get_data()
        y = epochs_clean.events[:, -1]
        
        # Normalize
        X_norm = (X - X.mean(axis=-1, keepdims=True)) / (X.std(axis=-1, keepdims=True) + 1e-6)

        save_name_npz = file_name.replace('.gdf', '_cleaned.npz')
        np.savez(os.path.join(OUTPUT_DIR, save_name_npz), X=X_norm, y=y)
        print(f"  -> Xong: {save_name_npz} | Shape: {X_norm.shape}")

    except Exception as e:
        print(f"  -> LOI: {e}")
        # In ra trace để debug nếu cần
        # import traceback
        # traceback.print_exc()

if __name__ == "__main__":
    gdf_files = glob.glob(os.path.join(DATA_DIR, '*.gdf'))
    for f in gdf_files:
        process_file(f)


Dang xu ly: A01E.gdf ...


D:\admin\Python310\lib\contextlib.py:142: RuntimeWarning: Channel names are not unique, found duplicates for: {'EEG'}. Applying running numbers for duplicates.
  next(self.gen)


  -> LOI: No matching events found for right_hand (event id 8)

Dang xu ly: A01T.gdf ...


D:\admin\Python310\lib\contextlib.py:142: RuntimeWarning: Channel names are not unique, found duplicates for: {'EEG'}. Applying running numbers for duplicates.
  next(self.gen)


  -> Xong: A01T_cleaned.npz | Shape: (144, 22, 501)

Dang xu ly: A02E.gdf ...


...

Dropped 1 epoch: 130
  -> Xong: A08T_cleaned.npz | Shape: (143, 22, 501)

Dang xu ly: A09E.gdf ...


D:\admin\Python310\lib\contextlib.py:142: RuntimeWarning: Channel names are not unique, found duplicates for: {'EEG'}. Applying running numbers for duplicates.
  next(self.gen)


  -> LOI: No matching events found for right_hand (event id 8)

Dang xu ly: A09T.gdf ...


D:\admin\Python310\lib\contextlib.py:142: RuntimeWarning: Channel names are not unique, found duplicates for: {'EEG'}. Applying running numbers for duplicates.
  next(self.gen)


Dropped 1 epoch: 97
  -> Xong: A09T_cleaned.npz | Shape: (143, 22, 501)


In [1]:
import os
import glob
import mne
import numpy as np
from autoreject import AutoReject

DATA_DIR = './data'
OUTPUT_DIR = './data2'
os.makedirs(OUTPUT_DIR, exist_ok=True)

# Danh sách tên kênh chuẩn của bộ dữ liệu BCI Competition IV 2a (22 kênh EEG + 3 EOG)
# Thứ tự này là cố định cho dataset này.
BCI_IV_2a_CHANNELS = [
    'Fz', 'FC3', 'FC1', 'FCz', 'FC2', 'FC4', 'C5', 'C3', 'C1', 'Cz', 
    'C2', 'C4', 'C6', 'CP3', 'CP1', 'CPz', 'CP2', 'CP4', 'P1', 'Pz', 
    'P2', 'POz', 'EOG-left', 'EOG-central', 'EOG-right'
]

def process_file(file_path):
    file_name = os.path.basename(file_path)
    print(f"\nDang xu ly: {file_name} ...")
    
    try:
        # 1. Load Data
        raw = mne.io.read_raw_gdf(file_path, preload=True, verbose=False)

        # --- BƯỚC FIX LỖI QUAN TRỌNG ---
        # Kiểm tra số lượng kênh. Dataset này thường có 25 kênh (22 EEG + 3 EOG)
        if len(raw.ch_names) == 25:
            # Tạo dictionary mapping tên cũ (bất kể là gì) sang tên chuẩn
            rename_dict = {old: new for old, new in zip(raw.ch_names, BCI_IV_2a_CHANNELS)}
            raw.rename_channels(rename_dict)
        else:
            print(f"Warning: Số lượng kênh không khớp (gốc: {len(raw.ch_names)}), bỏ qua rename chuẩn.")

        # Định nghĩa lại loại kênh (Set Channel Types)
        # 22 kênh đầu là EEG, 3 kênh cuối là EOG
        # Việc này giúp set_montage biết kênh nào cần tìm tọa độ, kênh nào không
        ch_types = {name: 'eeg' for name in BCI_IV_2a_CHANNELS[:22]}
        ch_types.update({name: 'eog' for name in BCI_IV_2a_CHANNELS[22:]})
        raw.set_channel_types(ch_types)

        # 3. Chọn kênh xử lý (Thay thế pick_types bằng pick như warning gợi ý)
        # Chỉ giữ lại EEG để lọc và xử lý, bỏ EOG đi
        raw.pick(['eeg']) 
        
        # 7. Epoching
        events, event_dict = mne.events_from_annotations(raw, verbose=False)
        
        # Mapping Event ID (769/770 là chuẩn BCI IV 2a gốc, 7/8 là file đã convert)
        # Ta check xem file dùng ID nào
        unique_events = np.unique(events[:, 2])
        if 7 in unique_events:
            event_id = {'left_hand': 7, 'right_hand': 8}
        elif 769 in unique_events:
            event_id = {'left_hand': 769, 'right_hand': 770}
        else:
            print(f"  -> Skip: Không tìm thấy event Left/Right. Events found: {unique_events}")
            return

        epochs = mne.Epochs(raw, events, event_id, tmin=0.5, tmax=2.5,
                            baseline=None, preload=True, event_repeated='drop', verbose=False)

        # 9. Lưu Data
        X = epochs.get_data()
        y = epochs.events[:, -1]

        save_name_npz = file_name.replace('.gdf', '.npz')
        np.savez(os.path.join(OUTPUT_DIR, save_name_npz), X=X, y=y)
        print(f"  -> Xong: {save_name_npz} | Shape: {X.shape}")

    except Exception as e:
        print(f"  -> LOI: {e}")
        # In ra trace để debug nếu cần
        # import traceback
        # traceback.print_exc()

if __name__ == "__main__":
    gdf_files = glob.glob(os.path.join(DATA_DIR, '*.gdf'))
    for f in gdf_files:
        process_file(f)


Dang xu ly: A01T.gdf ...


D:\admin\Python310\lib\contextlib.py:142: RuntimeWarning: Channel names are not unique, found duplicates for: {'EEG'}. Applying running numbers for duplicates.
  next(self.gen)


  -> Xong: A01T.npz | Shape: (144, 22, 501)

Dang xu ly: A02T.gdf ...


D:\admin\Python310\lib\contextlib.py:142: RuntimeWarning: Channel names are not unique, found duplicates for: {'EEG'}. Applying running numbers for duplicates.
  next(self.gen)


  -> Xong: A02T.npz | Shape: (144, 22, 501)

Dang xu ly: A03T.gdf ...


...

  -> Xong: A07T.npz | Shape: (144, 22, 501)

Dang xu ly: A08T.gdf ...


D:\admin\Python310\lib\contextlib.py:142: RuntimeWarning: Channel names are not unique, found duplicates for: {'EEG'}. Applying running numbers for duplicates.
  next(self.gen)


  -> Xong: A08T.npz | Shape: (144, 22, 501)

Dang xu ly: A09T.gdf ...


D:\admin\Python310\lib\contextlib.py:142: RuntimeWarning: Channel names are not unique, found duplicates for: {'EEG'}. Applying running numbers for duplicates.
  next(self.gen)


  -> Xong: A09T.npz | Shape: (144, 22, 501)
